# Install dependencies


In [ ]:
!pip install persiantools
import json
from google.colab import files
import pandas as pd
import io
from persiantools.jdatetime import JalaliDate
import datetime

# Upload Json file 


**Sample structure**
```
{
  "data": {
    "2022/01/01": "100",
    "2022/01/02": "150",
    "2022/01/03": "200",
    "2022/01/22": "800"
  },
  "config": {
    "type": "miladi",
    "time": "daily",
    "interpolation": "linear"
  }
}
```



In [ ]:
#choose your json file to be uploaded
uploaded = files.upload()

# Read each data and config from uploaded Json seprately into Pandas Dataframes





In [ ]:
# replace your file_name by your own file name. (ex: 'sample.json')
data = json.loads(uploaded[file_name].decode("utf-8"))

input_time_series = pd.DataFrame(data["data"].items(), columns=['Date', 'price'])

config = pd.DataFrame(data["config"].items())
config.columns=['params', 'option']
config = config.set_index('params')
print(input_time_series)
print(config)

## Convert Gregorian to Shamsi

In [ ]:
# function that get the data and convert
def to_jalali_date(x):
  year, month, day=x.split('-', 3)

  return GregorianDate(int(year), int(month), int(day)).to_jalali_date()

In [ ]:
date_type =  config.iloc[config.index=='type']['option'].values[0]
ts=input_time_series
if date_type == 'miladi':
  ts['Date'] = ts['Date'].apply(to_gregorian_date)
ts['Date'] = ts['Date'].astype('datetime64[ns]')
data = ts.sort_values(by=['Date'], ascending=[True])
ts = ts.set_index('Date')
ts['price']= ts['price'].astype('int')
print(ts)

# Generate the missing dates based on defined frequency

In [ ]:
time_type =  config.iloc[config.index=='time']['option'].values[0]

if time_type == 'daily':
  time_type = 'd'
elif time_type == 'monthly':
  time_type = 'm'
elif time_type == 'yearly':
  time_type = 'y'

ts = ts.asfreq(time_type)
print(ts)

1. *Interpolation*

# Interpolate the missing values 
(Bonus task) Diffrent type of interpolation can be run

In [ ]:
interpolation_type =  config.iloc[config.index=='interpolation']['option'].values[0]
print(interpolation_type)
def interpolation_f(input_data, Interpolation_type):
  if interpolation_type == 'linear':
    return input_data.interpolate(method='linear')
  elif interpolation_type == 'spline':
    return input_data.interpolate(method='spline', order=3)
  elif interpolation_type == 'polynomial':
    return input_data.interpolate(method='polynomial', order=2)
  else:
    return input_data


ts['price'] = interpolation_f(ts['price'], interpolation_type)
print(ts)

# Print the output JSON

In [ ]:
ts.set_index('Date')['price'].to_dict()